In [1]:
import clip
import torch
from PIL import Image

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Load the CLIP tokenizer
tokenizer = clip.tokenize

def image_caption(image_path):
    # Load and preprocess the image
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Encode the image
    with torch.no_grad():
        image_features = model.encode_image(image_input)

    # Prepare a textual prompt
    prompt = "A photo of"

    # Generate a caption using CLIP
    with torch.no_grad():
        text = clip.tokenize(prompt).to(device)
        text_features = model.encode_text(text)
        image_input_int = image_input.to(torch.long)  # Convert to torch.long
        logits_per_image, logits_per_text = model(image_input_int, text_features)
        probs = logits_per_text.softmax(dim=-1).cpu().numpy()

    # Find the most likely label
    label = " ".join(clip.tokenize(prompt).split(" ")[3:])
    label_probs = sorted(list(zip(model.vocab, probs.flatten())), key=lambda x: -x[1])

    # Return the most likely label
    return label_probs[0][0]

# Example usage
image_path = r"C:\Users\IDAC PC\Desktop\UtkuThesis\ImageCaptioning\tmp.jpg"
caption = image_caption(image_path)
print("Image Caption:", caption)


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.HalfTensor instead (while checking arguments for embedding)

In [4]:
import torch
import clip
from PIL import Image
torch.cuda.empty_cache()
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
image_path = r"C:\Users\IDAC PC\Desktop\UtkuThesis\ImageCaptioning\tmp.jpg"
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    # print(image_features)

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.3528 0.5015 0.1459]]
